In [1]:
# === MULTI-CURRENCY PROFIT-ONLY TRADING BOT (MT5 + Python) ===

import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import talib
import time

# === CONFIG ===
SYMBOLS = [
    "XAUUSD", "EURUSD", "GBPUSD", "USDJPY", "AUDUSD",
    "USDCAD", "NZDUSD", "USDCHF", "EURJPY", "GBPJPY",
    "EURGBP", "CADJPY", "CHFJPY", "AUDJPY", "EURNZD"
]
TIMEFRAME = mt5.TIMEFRAME_M15
LOT_SIZE = 0.1
MIN_PROFIT_USD = 0.01  # close trade when profit >= $1
MAX_TRADES = 5  # maximum concurrent trades
FORCE_MODE = True  # force trades even if no signal

# === INIT ===
if not mt5.initialize():
    raise RuntimeError(f"MT5 initialize() failed: {mt5.last_error()}")

account = mt5.account_info()
if account is None:
    raise RuntimeError(f"MT5 account_info() failed: {mt5.last_error()}")

for symbol in SYMBOLS:
    info = mt5.symbol_info(symbol)
    if info is None:
        raise RuntimeError(f"Symbol {symbol} not found")
    if not info.visible:
        mt5.symbol_select(symbol, True)

# === HELPER: Get recent candles ===
def get_data(symbol, timeframe, n=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n)
    if rates is None or len(rates) == 0:
        raise RuntimeError(f"Failed to retrieve market data for {symbol}: {mt5.last_error()}")
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# === HELPER: Entry Signal (Custom Strategy Per Symbol) ===
def get_trade_signal(df, symbol):
    close = df['close'].astype(float).values
    open_ = df['open'].astype(float).values
    high = df['high'].astype(float).values
    low = df['low'].astype(float).values

    rsi = talib.RSI(close, timeperiod=14)
    engulf = talib.CDLENGULFING(open_, high, low, close)

    if len(rsi) == 0 or len(engulf) == 0:
        return None

    last_rsi = rsi[-1]
    last_engulf = engulf[-1]
    print(f"{symbol} - RSI: {last_rsi:.2f}, Engulfing: {last_engulf}")

    # Custom strategy per symbol
    if symbol == "XAUUSD" and last_engulf > 0 and last_rsi < 45:
        return "buy"
    elif symbol == "XAUUSD" and last_engulf < 0 and last_rsi > 55:
        return "sell"
    elif symbol == "EURUSD" and last_engulf > 0 and last_rsi < 35:
        return "buy"
    elif symbol == "EURUSD" and last_engulf < 0 and last_rsi > 65:
        return "sell"
    elif last_engulf > 0 and last_rsi < 40:
        return "buy"
    elif last_engulf < 0 and last_rsi > 60:
        return "sell"
    return None

# === HELPER: Open Trade ===
def open_trade(symbol, direction):
    tick = mt5.symbol_info_tick(symbol)
    price = tick.ask if direction == "buy" else tick.bid
    order_type = mt5.ORDER_TYPE_BUY if direction == "buy" else mt5.ORDER_TYPE_SELL

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": LOT_SIZE,
        "type": order_type,
        "price": price,
        "deviation": 20,
        "magic": 10001,
        "comment": "profit_only_trade",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    if result.retcode == mt5.TRADE_RETCODE_DONE:
        print(f"Opened {direction.upper()} for {symbol} at {price}")
    else:
        print(f"Order failed for {symbol}: {result.retcode} - {getattr(result, 'comment', 'No comment')}")
        print(result._asdict())

# === HELPER: Close Trade if profit >= $1 ===
def check_and_close():
    for symbol in SYMBOLS:
        positions = mt5.positions_get(symbol=symbol)
        if positions:
            pos = positions[0]
            profit = pos.profit
            print(f"{symbol} position profit: ${profit:.2f}")
            if profit >= MIN_PROFIT_USD:
                close_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                price = mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).ask
                close_req = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": pos.volume,
                    "type": close_type,
                    "position": pos.ticket,
                    "price": price,
                    "deviation": 20,
                    "magic": 10001,
                    "comment": "close_in_profit",
                    "type_time": mt5.ORDER_TIME_GTC,
                    "type_filling": mt5.ORDER_FILLING_IOC,
                }
                result = mt5.order_send(close_req)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    print(f"Closed {symbol} in profit: ${profit:.2f}")
                else:
                    print(f"Close failed for {symbol}: {result.retcode}")

# === MAIN LOOP ===
print("Running multi-currency profit-only bot...")
try:
    while True:
        check_and_close()
        time.sleep(2)

        current_trades = len(mt5.positions_get() or [])
        if current_trades >= MAX_TRADES:
            print(f"Max trades ({MAX_TRADES}) reached. Waiting to open new trades.")
            time.sleep(60)
            continue

        for symbol in SYMBOLS:
            if not any(pos.symbol == symbol for pos in mt5.positions_get() or []):
                df = get_data(symbol, TIMEFRAME, 100)
                signal = get_trade_signal(df, symbol)
                if FORCE_MODE or signal:
                    open_trade(symbol, signal)

        time.sleep(60)  # wait 1 min before checking again
except KeyboardInterrupt:
    print("Stopped by user")
finally:
    mt5.shutdown()


RuntimeError: MT5 initialize() failed: (-10005, 'IPC timeout')